### Denoising fMRI Scans with Diffusion Models:

## Coregistering sMRI and fMRI

This code is for co-registering a structural MRI (sMRI) to the space of the functional MRI (fMRI) data.

This script performs the following steps for each subject:
1.  Loads the fMRI and sMRI data.
2.  Creates a mean image from the 4D fMRI data to use as a stable registration target.
3.  Resamples the sMRI to match the affine transformation and voxel grid of the
    mean fMRI. This single step effectively co-registers and resizes the image.
4.  Saves the co-registered sMRI.


______

In [ ]:
import os
import re
import nibabel as nib
from nilearn import image, plotting

def coregister_and_resample(fmri_path, smri_path, output_path, qc_plot_path=None):
    """
    Co-registers and resamples an sMRI to an fMRI.

    Args:
        fmri_path (str): Path to the 4D fMRI NIfTI file (.nii or .nii.gz).
        smri_path (str): Path to the 3D T1-weighted sMRI NIfTI file.
        output_path (str): Path to save the co-registered sMRI NIfTI file.
        qc_plot_path (str, optional): Path to save the quality control plot.
                                      If None, the plot is displayed interactively.
                                      Defaults to None.

    Returns:
        str: The path to the co-registered sMRI file.
    """
    print(f"\n--- Starting Co-registration for: {os.path.basename(fmri_path)} ---")

    # load data
    fmri_img = image.load_img(fmri_path)
    smri_img = image.load_img(smri_path)

    # mean fMRI from 4D
    mean_fmri_img = image.mean_img(fmri_img)

    
    #co-register and resample the sMRI to the fMRI space
    smri_coreg_resampled = image.resample_to_img(
        source_img=smri_img,
        target_img=mean_fmri_img,
        interpolation='continuous'
    )

    # save
    print(f"Saving co-registered sMRI to: {output_path}")
    smri_coreg_resampled.to_filename(output_path)
    print("File saved successfully.")




if __name__ == '__main__':
    # Define the base directory for outputs
    base_output_dir = '/kaggle/working/coregistered_outputs'
    os.makedirs(base_output_dir, exist_ok=True)
    
    # List of paths for the fMRI files
    fmri_files = [
        '/kaggle/input/fmri-data/sub-01_ses-1_task-motor_run-10_bold.nii',
        '/kaggle/input/fmri-data/sub-dd_ses-1_task-motor_run-10_bold.nii',
        '/kaggle/input/fmri-data/sub-gg_ses-1_task-motor_run-10_bold.nii',
        '/kaggle/input/fmri-data/sub-uu_ses-1_task-motor_run-10_bold.nii'
    ]

    # List of paths for the corresponding anatomical files (in matching order)
    smri_files = [
        '/kaggle/input/anatomy-data/sub-01_ses-1_T1w.nii/sub-01_ses-1_T1w.nii',
        '/kaggle/input/anatomy-data/sub-dd_ses-1_T1w.nii/sub-dd_ses-1_T1w.nii',
        '/kaggle/input/anatomy-data/sub-gg_ses-1_T1w.nii/sub-gg_ses-1_T1w.nii',
        '/kaggle/input/anatomy-data/sub-uu_ses-1_T1w.nii/sub-uu_ses-1_T1w.nii' 
    ]

    # zip() pairs each fMRI file with its corresponding sMRI file.
    for fmri_path, smri_path in zip(fmri_files, smri_files):
        
        # check if input files exist before running
        if not os.path.exists(fmri_path) or not os.path.exists(smri_path):
            print(f"\nERROR: Skipping pair due to missing files.")
            if not os.path.exists(fmri_path):
                print(f"  - fMRI not found at: {fmri_path}")
            if not os.path.exists(smri_path):
                print(f"  - sMRI not found at: {smri_path}")
            continue # Skip to the next pair

        # Generate Dynamic Output Paths
        match = re.search(r'(sub-[^_]+)', os.path.basename(fmri_path))
        if match:
            subject_id = match.group(1)
        else:
            subject_id = os.path.basename(fmri_path).split('.')[0]

        output_smri_coreg_path = os.path.join(base_output_dir, f'{subject_id}_smri_coregistered.nii.gz')
        output_qc_plot_path = os.path.join(base_output_dir, f'{subject_id}_coregistration_qc.png')

        # run the co-registration process
        try:
            coregister_and_resample(
                fmri_path=fmri_path,
                smri_path=smri_path,
                output_path=output_smri_coreg_path,
                qc_plot_path=output_qc_plot_path
            )
        except Exception as e:
            print(f"\nAN ERROR OCCURRED while processing {subject_id}: {e}")
            print("Continuing to the next subject...")

    print(base_output_dir)